![ss](images/Spark核心RDD.jpg)

    1.PARTITION是RDD的基本单位,不同的处理阶段产生了依赖关系的PARTITION,依赖关系体现了SPARK的弹性,并且参与数据的容错恢复.
    2.各个PARTITION的处理是并行的,一个PARTITION只能对于一个TASK,由一个核心处理
    3.从一种PARTITION,变成另一种PARTITION的操作称为一个算子!
    4.Partition可以分布在多个节点上

## standalone模式下面的Spark执行原理
![ss](images/spark_standalone.png)

    1.Driver是standalone模式下的 任务 主节点,一个JVM进程
    2.Worker是standalone模式下的 从节点,一个JVM进程
    3.Driver发放任务到worker,汇集worker的计算结果,谨防OOM
    4,master是集群的管理者,汇集集群中的资源(worker信息)
   

## 基于standalone资源管理 集群搭建
    standalone是spark自带的资源调度框架
    
    1.conf/slaves:加入node2,node3,node4
    2.conf/spark-env.sh:修改如下变量
        export SPARK_MASTER_IP=node1
        export SPARK_MASTER_PORT=7077
        export SPARK_WORKER_CORES=2
        export SPARK_WORKER_MEMORY=2g
        export SPARK_MASTER_WEBUI_PORT=8888
        //主MasterIP,port
        //每个worker使用2核心,2G内存
        //web接口是8888
    3.在主master 启动全部节点:
        sbin/start-all.sh
        sbin/stop-all.sh
        http://node1:8888
    4.提交作业
        有2种方式:
        //客户端方式,你的客户端就是driver!
        bin/spark-submit --master spark://node1:7077 --class main.spark.app.SparkPI myzxk.jar
        //集群方式,集群中一个worker被认为是driver
        bin/spark-submit --deploy-mode cluster --master spark://node1:7077 --class main.spark.app.SparkPI myzxk.jar
        
        (org.apache.spark.examples.SparkPi)
        $SPARK_HOME/bin/spark-submit --master spark://node1:7077 --class org.apache.spark.examples.SparkPi $SPARK_HOME/lib/spark-examples-1.6.0-hadoop2.6.0.jar

![](images/Standalone-client.jpg)

## 备注: Driver和worker进行大量的信息交互,汇集worker的计算结果,比如,RDD.collect(),把Work的数据拉到Driver,所有会造成client网络拥堵

![](images/Standalone-cluster.jpg)
## 集群模式,Driver是在集群中的随机一台机器上启动,所以客户端不会有网络拥堵的压力

## 基于yarn资源调度管理

    A.集群配置
    注意这里是新文件,要把之前配置standalone的删除!!!
    
        conf/spark-env.sh  或者/conf/spark-default.conf:修改如下变量,
        HADOOP_CONF_DIR=HADOOP_HOME/etc/hadoop
    B.yarn资源调度处理任务的流程
        回顾下map-reduce,yarn调度:
        1.yarn构架 分为resourceManager,NodeManager,resourceManager协调,管理,监控NodeManager,NodeManager节点
        上面跑实际的 任务
        2.job被提交给resourceManager后,resourceManager随机挑选一台NodeManager,创建APP-MASTER(可以认为是java对象),
        app_master 创建,跟踪,作业
        3.app_master要求resourceManager,在NodeManager上分配资源,启动container(JVM),一个大job就这样在各个NodeManager
        节点的container(JVM)运行
    C.yarn-client的执行
        与map-reduce的区别:客户端是driver,app_master要求申请的不是container,而是executor,executor.threadpool与driver
        频繁的通信,结果collect 给客户端driver
        client-->ResourceManager-->applicationMaster-->申请创建executor-->executor.threadpool
            |                                                                        | 
            |                                                                        |
            driver<------------------------------------------------------------------>|
        命令:bin/spark-submit --master yarn --deploy-mode client --class main.spark.app.SparkPI myzxk.jar
    D.yarn-cluster的执行:
        与yarn-client的区别,client不再是driver,applicationMaster=driver
        命令:bin/spark-submit --master yarn --deploy-mode cluster --class main.spark.app.SparkPI myzxk.jar
    E.在node1:8088/任务的history 下面的logs看结果
  ![ss](images/Yarn-client模式提交任务.jpg)
  ![ss](images/Yarn-cluster模式提交任务.jpg)

## RDD的宽窄模式
![ss](images/RDD的宽窄依赖.jpg)

        SPARK依据宽依赖把JOB划分成多个stage,stages组成DAG,每个stage由于操作RDD有partiion,所有可以划分成多个tasks,task是driver交给worker的
        executor执行的基本单位, 换言之,有了partiion,所有有了并行管道执行,任务分为各个部分(STAGE)->>DAG,执行需要顺序!

## 资源调度与任务调度(代码层)
![ss](images/Spark资源调度和任务调度.jpg)

    1.SparkContext是客户端创建的,SparkContext=DAGScheduler+TaskScheduler
    2.TaskScheduler主要是资源调度,DAGScheduler任务调度
    3.spark集群什么都不知道,只知道完成TaskScheduler发来的一个一个TASK,只有客户端知道要执行任务的"全部面貌",
    DAGScheduler把任务划分为不同stage,按照顺序执行DAG
    
    4.SPARK的任务调度: 粗力度:
        A).APPLICATION启动前,计算好需要的全部tasks的全部资源,在sparkconf已经配置好了,给clusterManager(standalone|yarn),资源一旦满足,马上执行任务
        B).直到所有task执行完毕后,才释放资源, 如果不是所有TASK完成,那就 "有借不还了"
    5.MR细调度: 任务调来:
        APP-MASTER在节点创建TASKMANAGER,TASKMANAGER复制一个TASK的执行,执行完成就释放
        C)对比:MR 调度过于琐碎,但是资源利用充分,SPARK集中资源调度,资源回收,可以达到加速计算的效果,但是由于统一管理
        原因,可能会造成资源被过度占用的浪费!